<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Image-Retrieval/FaceExtraction_customModel_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import cv2
import dlib
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [14]:
# Create output directory if not exists
output_folder = "extracted_faces"
os.makedirs(output_folder, exist_ok=True)

In [15]:
# Load image
def detect_and_save_faces(image_path):
    detector = dlib.get_frontal_face_detector()
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    face_images = []
    for i, face in enumerate(faces):
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_img = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (100, 100))  # Resizing to the required input size

        # Ensure the face image is in the correct format (grayscale, 1 channel)
        face_resized = np.expand_dims(face_resized, axis=-1)  # Adding the channel dimension

        face_path = os.path.join(output_folder, f"face_{i}.jpg")
        cv2.imwrite(face_path, face_resized)  # Save the face as grayscale image
        face_images.append(face_resized)

    return face_images

In [16]:
# Train a simple CNN on extracted faces
def build_face_recognition_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Assuming two classes for demo
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# Prepare dataset for training
def prepare_data():
    face_images = []
    labels = []

    # Loop through all files in the directory
    for filename in os.listdir(output_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Check if it's an image file
            face_path = os.path.join(output_folder, filename)
            face_img = cv2.imread(face_path, cv2.IMREAD_GRAYSCALE)
            face_resized = cv2.resize(face_img, (100, 100))  # Resize to model input size
            face_resized = np.expand_dims(face_resized, axis=-1)  # Add channel dimension (grayscale)

            face_images.append(face_resized)
            # Here we assume labels for two classes (0 and 1), you need to update based on your use case
            label = 0  # Update with appropriate label for your case
            labels.append(label)

    # Convert to numpy arrays
    face_images = np.array(face_images)
    labels = np.array(labels)

    # Split into training and validation sets
    train_paths, val_paths, train_labels, val_labels = train_test_split(face_images, labels, test_size=0.2, random_state=42)
    return train_paths, val_paths, train_labels, val_labels

if __name__ == "__main__":
    # Change this to your actual group photos directory
    group_photos_dir = "/content/drive/MyDrive/Group-Photos"

    # Loop through all files in the directory
    for filename in os.listdir(group_photos_dir):
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Check if it's an image file
            image_path = os.path.join(group_photos_dir, filename)
            detect_and_save_faces(image_path)

    # Prepare data for training and validation
    train_data, val_data, train_labels, val_labels = prepare_data()

    # Initialize and train the model
    model = build_face_recognition_model()

    # Train the model
    model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10)

    # Save the model
    model.save("face_recognition_model.h5")
    print("Model trained and saved successfully.")

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 18.9935 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 685ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 888ms/step - accuracy: 1.

Model trained and saved successfully.
